## import

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import sqlite3

## スクレイピング → repo_data

In [2]:
# Google 組織のリポジトリ一覧ページからリンクを取得
url = "https://github.com/orgs/google/repositories"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

repos = soup.select('h3 a[href^="/google/"]')
print("取得したリポジトリ数:", len(repos))

repo_data = []

for repo in repos[:10]:
    name = repo.get_text(strip=True)
    href = repo["href"]
    repo_url = "https://github.com" + href

    print("\n--- アクセス中:", name, repo_url)

    r = requests.get(repo_url, headers=headers)
    s = BeautifulSoup(r.text, "html.parser")

    # 主要言語
    lang_tag = s.find("span", {"itemprop": "programmingLanguage"})
    lang = lang_tag.get_text(strip=True) if lang_tag else "Unknown"

    # スター数
    star_link = s.find("a", href=lambda h: h and h.endswith("/stargazers"))
    if star_link:
        stars_text = star_link.get_text(strip=True)
        stars_text = stars_text.replace(",", "").replace("stars", "").strip()
        if stars_text.lower().endswith("k"):
            stars = int(float(stars_text[:-1]) * 1000)
        else:
            try:
                stars = int(float(stars_text))
            except ValueError:
                stars = 0
    else:
        stars = 0

    print("  主要言語:", lang)
    print("  スター数:", stars)

    repo_data.append((name, lang, stars))
    time.sleep(1)

取得したリポジトリ数: 30

--- アクセス中: flatbuffers https://github.com/google/flatbuffers
  主要言語: Unknown
  スター数: 25100

--- アクセス中: dwh-migration-tools https://github.com/google/dwh-migration-tools
  主要言語: Unknown
  スター数: 54

--- アクセス中: perfetto https://github.com/google/perfetto
  主要言語: Unknown
  スター数: 5000

--- アクセス中: nearby https://github.com/google/nearby
  主要言語: Unknown
  スター数: 887

--- アクセス中: docsy https://github.com/google/docsy
  主要言語: Unknown
  スター数: 2900

--- アクセス中: googletest-rust https://github.com/google/googletest-rust
  主要言語: Unknown
  スター数: 393

--- アクセス中: site-kit-wp https://github.com/google/site-kit-wp
  主要言語: Unknown
  スター数: 1300

--- アクセス中: orbax https://github.com/google/orbax
  主要言語: Unknown
  スター数: 455

--- アクセス中: angle https://github.com/google/angle
  主要言語: Unknown
  スター数: 3800

--- アクセス中: osv-scalibr https://github.com/google/osv-scalibr
  主要言語: Unknown
  スター数: 537


## SQLite に保存 → SELECT 文で表示

In [3]:
path = ''
db_name = 'github_google.db'

try:
    conn = sqlite3.connect(path + db_name)
    cur = conn.cursor()

    sql = """
    CREATE TABLE IF NOT EXISTS repos (
        id    INTEGER PRIMARY KEY AUTOINCREMENT,
        name  TEXT,
        lang  TEXT,
        stars INTEGER
    );
    """
    cur.execute(sql)
    conn.commit()

    insert_sql = "INSERT INTO repos (name, lang, stars) VALUES (?, ?, ?);"
    cur.executemany(insert_sql, repo_data)
    conn.commit()

    print("=== DB に保存されたデータ ===")
    cur.execute("SELECT id, name, lang, stars FROM repos")

    for row in cur:
        print(row)

except sqlite3.Error as e:
    print("エラーが発生しました:", e)

finally:
    conn.close()

=== DB に保存されたデータ ===
(1, 'flatbuffers', 'Unknown', 25100)
(2, 'dwh-migration-tools', 'Unknown', 54)
(3, 'perfetto', 'Unknown', 5000)
(4, 'nearby', 'Unknown', 887)
(5, 'docsy', 'Unknown', 2900)
(6, 'googletest-rust', 'Unknown', 393)
(7, 'site-kit-wp', 'Unknown', 1300)
(8, 'orbax', 'Unknown', 455)
(9, 'angle', 'Unknown', 3800)
(10, 'osv-scalibr', 'Unknown', 537)
